In [75]:
import akshare as ak
import pandas as pd
import matplotlib.pyplot as plt
import talib as ta

df = ak.stock_zh_a_daily(symbol="sh600570", start_date="19910403", end_date="20240417", adjust="hfq")

In [161]:
df['ret']=df.close/df.close.shift(1)-1
# high,low,close,amount = df.high.values,df.low.values,df.close.values,df.amount.values
# df['mfi']=ta.MFI(high,low,close,amount,14)
df['mfi']=ta.MFI(df.high,df.low,df.close,df.amount,14)
L=20
H=80
for i in range(14,len(df)):
    if df['mfi'][i]>L and df['mfi'][i-1]<L:
        df.loc[df.index[i],'收盘信号']=1
    if df['mfi'][i]<H and df['mfi'][i-1]>H:
        df.loc[df.index[i],'收盘信号']=0
df['当天仓位']=df['收盘信号'].shift(1)
df['当天仓位'].ffill(inplace=True)
d = df[df['收盘信号']==1].index[0]
df_new = df[d:]
df_new['策略净值']=(df_new['ret']*df_new['当天仓位']+1.0).cumprod()
df_new['指数净值']=df_new['close']/df_new['close'].iloc[0]
# df_new.to_excel("tmp.xlsx")

C:\Users\hspcadmin\AppData\Local\Temp\ipykernel_37800\1800615990.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['当天仓位'].ffill(inplace=True)
C:\Users\hspcadmin\AppData\Local\Temp\ipykernel_37800\1800615990.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['策略净值']=(df_ne

In [142]:
total_ret=df_new[['策略净值','指数净值']].iloc[-1]-1
annual_ret=pow(1+total_ret,250/len(df_new))-1

策略净值    0.060821
指数净值    0.253267
Name: 4834, dtype: float64
4697


In [168]:
df_new['策略收益率']=df_new['策略净值']/df_new['策略净值'].shift(1)-1
df_new['指数收益率']=df_new.ret
# beta=df_new[['策略收益率','ret']].cov().iat[0,1]/df_new['ret'].var()
# beta
total_ret=df_new[['策略净值','指数净值']].iloc[-1]-1
annual_ret=pow(1+total_ret,250/len(df_new))-1
beta=df_new[['策略收益率','指数收益率']].cov().iat[0,1]/df_new['指数收益率'].var()
alpha=(annual_ret['策略净值']-annual_ret['指数净值']*beta)
alpha

C:\Users\hspcadmin\AppData\Local\Temp\ipykernel_37800\633446065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['策略收益率']=df_new['策略净值']/df_new['策略净值'].shift(1)-1
C:\Users\hspcadmin\AppData\Local\Temp\ipykernel_37800\633446065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['指数收益率']=df_new.ret


0.0038578268555360326

In [176]:
import numpy as np
exReturn=df_new['策略收益率']-0.03/250
print(exReturn)
sharper_atio=np.sqrt(len(exReturn))*exReturn.mean()/exReturn.std()
sharper_atio

138          NaN
139    -0.023789
140    -0.008453
141     0.016687
142    -0.036183
          ...   
4830   -0.000120
4831   -0.000120
4832   -0.000120
4833   -0.026756
4834    0.027737
Name: 策略收益率, Length: 4697, dtype: float64


1.0400710680926346